## Gemini 1.5 pro chat with SQL

### This notebook demonstrates how to use Gemini 1.5 pro, a powerful language model, to interact with SQL databases. We'll set up the environment, initialize the Gemini 1.5 pro model, and then use it to generate SQL queries and natural language responses based on a database schema. This combination of LLM and SQL capabilities allows for more intuitive database interactions and insights.


In [2]:
! pip install langchain langchain-google-genai


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\bang0\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Language Model (LLM) Setup: Gemini 1.5 Pro

### This section outlines the setup for Gemini 1.5 Pro, a powerful language model developed by Google.

### 1. API Access:
####    - We'll use the Google AI API to access Gemini 1.5 Pro.
####    - An API key is required, which we'll securely input using getpass.

### 2. Key Features of Gemini 1.5 Pro:
####    - Advanced natural language understanding and generation
####    - Ability to handle complex, multi-turn conversations
####    - Proficiency in various tasks, including code generation and analysis

### 3. Integration with SQL:
####    - Gemini 1.5 Pro can generate SQL queries based on natural language input
####    - It can also provide natural language explanations of SQL results

### This setup allows for powerful, AI-driven database interactions, combining the strengths of 
### a state-of-the-art language model with structured data querying capabilities.

In [3]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

## DB

Connect to a SQLite DB.

To create this particular DB, you can use the code and follow the steps shown [here](https://github.com/facebookresearch/llama-recipes/blob/main/demo_apps/StructuredLlama.ipynb).

In [14]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///nba_roster.db", sample_rows_in_table_info=0)


def get_schema(_):
    return db.get_table_info()


def run_query(query):
    return db.run(query)


## Query a SQL Database 

Follow the runnables workflow [here](https://python.langchain.com/docs/expression_language/cookbook/sql_db).

In [6]:
# Prompt
from langchain_core.prompts import ChatPromptTemplate

# Update the template based on the type of SQL Database like MySQL, Microsoft SQL Server and so on
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Given an input question, convert it to a SQL query. No pre-amble."),
        ("human", template),
    ]
)

# Chain to query
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def clean_sql_query(query):
    # Remove backticks, 'sql' identifier, and strip whitespace
    return query.replace('```sql', '').replace('```', '').strip()

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
    | clean_sql_query
)

sql_response.invoke({"question": "What team is Klay Thompson on?"})

"SELECT team FROM players WHERE name = 'Klay Thompson';"

We can review the results:

* [LangSmith trace](https://smith.langchain.com/public/afa56a06-b4e2-469a-a60f-c1746e75e42b/r) LLaMA2-13 Replicate API
* [LangSmith trace](https://smith.langchain.com/public/2d4ecc72-6b8f-4523-8f0b-ea95c6b54a1d/r) LLaMA2-13 local 


In [15]:
# Chain to answer
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question and SQL response, convert it to a natural language answer. No pre-amble.",
        ),
        ("human", template),
    ]
)

full_chain = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | llm
)

full_chain.invoke({"question": "How many unique teams are there?"})

AIMessage(content='There are no unique teams. \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-c660c728-47cb-4cce-b305-717c7372043d-0', usage_metadata={'input_tokens': 117, 'output_tokens': 6, 'total_tokens': 123})

We can review the results:

* [LangSmith trace](https://smith.langchain.com/public/10420721-746a-4806-8ecf-d6dc6399d739/r) LLaMA2-13 Replicate API
* [LangSmith trace](https://smith.langchain.com/public/5265ebab-0a22-4f37-936b-3300f2dfa1c1/r) LLaMA2-13 local 

## Chat with a SQL DB 

Next, we can add memory.

In [18]:
# Prompt
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

template = """Given an input question, convert it to a SQL query. No pre-amble. Based on the table schema below, write a SQL query that would answer the user's question:
{schema}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

# Chain to query with memory
from langchain_core.runnables import RunnableLambda

sql_chain = (
    RunnablePassthrough.assign(
        schema=get_schema,
        history=RunnableLambda(lambda x: memory.load_memory_variables(x)["history"]),
    )
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
    | clean_sql_query
)


def save(input_output):
    output = {"output": input_output.pop("output")}
    memory.save_context(input_output, output)
    return output["output"]


sql_response_memory = RunnablePassthrough.assign(output=sql_chain) | save
sql_response_memory.invoke({"question": "What team is Klay Thompson on?"})

'-- This database contains no information about players, only teams.\n-- Returning the teams table as a placeholder. \n-- Add player information to this database to provide more specific results.\nSELECT\n  *\nFROM teams;'

In [20]:
# Chain to answer
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question and SQL response, convert it to a natural language answer. No pre-amble.",
        ),
        ("human", template),
    ]
)
def extract_content(message):
    return message.content if hasattr(message, 'content') else str(message)

full_chain = (
    RunnablePassthrough.assign(query=sql_response_memory)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | llm
    | extract_content 
    | clean_sql_query
)

full_chain.invoke({"question": "What is his salary?"})

"This database doesn't have any information about salaries, only teams."

Here is the [trace](https://smith.langchain.com/public/54794d18-2337-4ce2-8b9f-3d8a2df89e51/r).